In [1]:
from collections import Counter
from tqdm import tqdm

In [2]:
EMPTY_TOKEN = "-"
TOKENS = ("X", "O", EMPTY_TOKEN)

In [3]:
def is_board_ended(board):
    """Check if the game has ended (win or draw)."""
    n = len(board)
    win_len = 3 if n == 3 else 5
    
    def check_line(line):
        count = 1
        prev = None
        for token in line:
            if token == EMPTY_TOKEN:
                prev = None
                count = 0
                continue
            if token == prev:
                count += 1
                if count >= win_len:
                    return True
            else:
                prev = token
                count = 1
        return False
    
    # Check rows
    for i in range(n):
        if check_line(board[i]):
            return True
    
    # Check columns
    for i in range(n):
        if check_line([board[j][i] for j in range(n)]):
            return True
    
    # Check diagonals
    for p in range(n * 2 - 1):
        diag1 = [board[i][p - i] for i in range(max(0, p - n + 1), min(p + 1, n)) if 0 <= p - i < n]
        if len(diag1) >= win_len and check_line(diag1):
            return True
    
    for p in range(-n + 1, n):
        diag2 = [board[i][i - p] for i in range(max(0, p), min(n, n + p)) if 0 <= i - p < n]
        if len(diag2) >= win_len and check_line(diag2):
            return True
    
    return False

In [4]:
def load_unique_boards(filepath, board_size=3):
    """
    Load unique board hashes from file, remove duplicates,
    and convert each to 2D list representation.
    """
    with open(filepath, "r") as f:
        lines = [line.strip() for line in f if line.strip()]

    unique = list(set(lines))
    print(f"Loaded {len(lines)} lines, {len(unique)} unique boards.")

    boards = []
    for hash_str in unique:
        if len(hash_str) != board_size * board_size:
            raise ValueError(f"Invalid board length for {hash_str}")
        board = [list(hash_str[i:i + board_size]) for i in range(0, len(hash_str), board_size)]
        boards.append(board)

    return boards


def count_winning_boards(filepath, board_size=3):
    """
    Count how many boards are ended (win detected),
    showing progress with tqdm.
    """
    boards = load_unique_boards(filepath, board_size)
    win_count = 0

    print("Checking boards for wins...")
    for board in tqdm(boards, desc="Processing boards", unit="board"):
        if is_board_ended(board):
            win_count += 1

    print(f"\nTotal unique boards: {len(boards)}")
    print(f"Winning boards: {win_count}")
    print(f"Non-winning boards: {len(boards) - win_count}")

    return win_count

In [ ]:
count_winning_boards("unique.out", board_size=5)

Loaded 41125000 lines, 41125000 unique boards.
